# Exemplo 01

## Conexão

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/15 18:38:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

## Exemplo de RDD Simples

In [4]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
print(type(rdd))

<class 'pyspark.rdd.RDD'>


In [5]:
print(rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Exemplo de RDD com Arquivo

In [6]:
rdd = sc.textFile('teste.txt')
print(type(rdd))

<class 'pyspark.rdd.RDD'>


In [12]:
print(rdd.collect())

['Apache Spark é um mecanismo de análise unificado para processamento de dados em grande escala. ', 'Fornece APIs para linguagens de alto nível como Java, Scala, Python e R', 'Permite um mecanismo otimizado para suporte a gráficos de execução geral. ', 'Possui um excelente conjunto de ferramentas, tais como: ', '    Spark SQL para SQL e processamento de dados estruturados;', '    MLlib para aprendizado de máquina;', '    GraphX para processamento de gráfico; e', '    Streams estruturados para computação incremental e processamento de fluxo.']


In [8]:
print(rdd.count())

8


In [9]:
print(sc.getConf().getAll())

[('spark.app.startTime', '1689446298048'), ('spark.executor.memory', '512m'), ('spark.app.submitTime', '1689446297917'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-notebook'), ('spark.app.id', 'app-20230715183819-0002'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.securit

In [13]:
import re

In [14]:
rdd_sem_pontuacao = rdd.map(lambda linha: re.sub(r'[^\w\s]', '', linha))
for linha in rdd_sem_pontuacao.collect():
    print(linha)

Apache Spark é um mecanismo de análise unificado para processamento de dados em grande escala 
Fornece APIs para linguagens de alto nível como Java Scala Python e R
Permite um mecanismo otimizado para suporte a gráficos de execução geral 
Possui um excelente conjunto de ferramentas tais como 
    Spark SQL para SQL e processamento de dados estruturados
    MLlib para aprendizado de máquina
    GraphX para processamento de gráfico e
    Streams estruturados para computação incremental e processamento de fluxo


In [17]:
# Definir a lista de palavras indesejadas
palavras_indesejadas = ["e", "é", "um", "para", "em", "como", "a", "tais", "como", "de", ""]

# Dividir cada linha em palavras e criar um RDD com todas as palavras
palavras = rdd_sem_pontuacao.flatMap(lambda linha: linha.split(" "))

# Filtrar as palavras indesejadas
palavras_filtradas = palavras.filter(lambda palavra: palavra not in palavras_indesejadas)

# Contar as ocorrências de cada palavra
contagem_palavras = palavras_filtradas.countByValue()

# Imprimir as palavras únicas e suas contagens
print("Palavras únicas encontradas e suas contagens:")
for palavra, contagem in contagem_palavras.items():
    print(palavra, "->", contagem)

Palavras únicas encontradas e suas contagens:
Apache -> 1
Spark -> 2
mecanismo -> 2
análise -> 1
unificado -> 1
processamento -> 4
dados -> 2
grande -> 1
escala -> 1
Fornece -> 1
APIs -> 1
linguagens -> 1
alto -> 1
nível -> 1
Java -> 1
Scala -> 1
Python -> 1
R -> 1
Permite -> 1
otimizado -> 1
suporte -> 1
gráficos -> 1
execução -> 1
geral -> 1
Possui -> 1
excelente -> 1
conjunto -> 1
ferramentas -> 1
SQL -> 2
estruturados -> 2
MLlib -> 1
aprendizado -> 1
máquina -> 1
GraphX -> 1
gráfico -> 1
Streams -> 1
computação -> 1
incremental -> 1
fluxo -> 1


## Pipeline

In [18]:
import random

In [19]:
def valorPressao(x):
    idt = x
    pas = random.randint(11, 19)
    pad = random.randint(70, 120)
    return (x, pas, pad)

In [20]:
rdd_pipe = sc.parallelize(range(1000)).map(valorPressao)
print(type(rdd_pipe))

<class 'pyspark.rdd.PipelinedRDD'>


In [21]:
rdd_pipe.take(10)

[(0, 17, 92),
 (1, 18, 102),
 (2, 19, 100),
 (3, 18, 78),
 (4, 12, 94),
 (5, 11, 89),
 (6, 12, 82),
 (7, 14, 79),
 (8, 18, 100),
 (9, 15, 76)]

In [22]:
df = rdd_pipe.toDF(("Id", "PAS", "PAD"))
type(df)

pyspark.sql.dataframe.DataFrame

In [23]:
df.show()

+---+---+---+
| Id|PAS|PAD|
+---+---+---+
|  0| 19|100|
|  1| 14| 89|
|  2| 11| 80|
|  3| 14|102|
|  4| 14|110|
|  5| 19| 95|
|  6| 16| 78|
|  7| 11| 93|
|  8| 13| 99|
|  9| 13| 91|
| 10| 14|103|
| 11| 19|117|
| 12| 12| 92|
| 13| 13| 97|
| 14| 11| 74|
| 15| 13| 95|
| 16| 12| 93|
| 17| 16| 85|
| 18| 12| 78|
| 19| 17| 88|
+---+---+---+
only showing top 20 rows



## Finalizar a seção

In [24]:
spark.sparkContext.stop()